# Infer SOMA - part 2

--------------

## OBJECTIVE:  Infer sub-cellular component #2: SOMA  in order to understand interactome 

To measure shape, position, and size of the cell body -- the soma.    There are a variety of signals from which we could make this inference.  The two most promising are a composite signal including the residual from linear unmixing (`ch = [1, 4, 5,7]`) and a signal derived from the lysosome channel (`ch = 1`).

Dependencies:
The CYTOSOL inference rely on the NUCLEI AND SOMA inference.  Therefore all of the sub-cellular objects rely on this segmentation.





# IMPORTS

In [1]:

# this needs to be organzied to explain the imports
from pathlib import Path
import os
from re import S

import numpy as np

import scipy
from scipy import ndimage as ndi

import napari

# function for core algorithm
import aicssegmentation

from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning
from aicssegmentation.core.MO_threshold import MO

from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice

from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage import morphology

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
# from aicsimageio import AICSImage

from napari.utils.notebook_display import nbscreenshot

#import .infer_subc.base
from infer_subc.base import *


--------------

# IMAGE PROCESSING Objective 2:  infer SOMA
  [OUTLINE: Objective #2](#summary-of-objectives)
## summary of steps

INPUT
- channel  1,4,5, and 7
- labeled NUCLEI (objective #1)

PRE-PROCESSING
-  scale to max 1.0
- gaussian  Filter window 10

CORE-PROCESSING
  - watershed from NU, global threshold, minimum cross-entropy
  - threshold smoothing scale: 1 pixel
  - threshold correction factor: .5 (more lenient)
  - lower / upper bounds  (0,1)
  - log transformed thresholding
  - fill holes
    - discard objects on borde
    - fill holes

- POST-PROCESSING
  - fill holes
  - remove small objects
  - keep only the "most intense" Soma


OUTPUT
- mask of SOMA
- mask of NU (contained by SOMA)

In [2]:

data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")
czi_img_folder = data_path/"raw"

list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
print(img_file_list[5])

test_img_name = img_file_list[5]

img_data, meta_dict = read_input_image(test_img_name)

raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/Users/ahenrie/Projects/Imaging/mcz_subcell/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [3]:
chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'NU_object'

#NU_object_filen = export_ome_tiff(NU_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

NU_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )

NU_labels = label(NU_object)


In [4]:

###################
# INPUT
###################

# composite_channels = [1,4,5,7]
composite_channels = [1,2,3,4] # andy's best guess but maybe need to scale channel 1 (lysosomes) mega
# #composite_channels = range(img_data.shape[0]-1)
composite_channels = [1]



gaussian_smoothing_sigma = 8
gaussian_smoothing_truncate_range = 3.0

gaussian_smoothing_sigma_3D = [gaussian_smoothing_sigma*scale[2]/x  for x in scale]



print(f"gaussian filtering width (2D) is ~ : { scale[1]*gaussian_smoothing_sigma} microns, scale:{scale}")
print(f"gaussian filtering width (3d) is ~ : {gaussian_smoothing_sigma_3D} pixels; :{[x*y for x,y in zip(scale,gaussian_smoothing_sigma_3D)]} microns")

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(img_data[1,:,:,:]) #  [0.0, 8]

gaussian filtering width (2D) is ~ : 0.6389732147869854 microns, scale:(0.5804527163320905, 0.07987165184837318, 0.07987165184837318)
gaussian filtering width (3d) is ~ : [1.1008187175429014, 8.0, 8.0] pixels; :[0.6389732147869854, 0.6389732147869854, 0.6389732147869854] microns
mean intensity of the stack: 641.5999045901829
the standard deviation of intensity of the stack: 2144.942904845627
0.9999 percentile of the stack intensity is: 49648.38039997965
minimum intensity of the stack: 0
maximum intensity of the stack: 65535
suggested upper range is 23.0, which is 49975.2867160396
suggested lower range is 0.0, which is 641.5999045901829
So, suggested parameter for normalization is [0.0, 23.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value


# WORKFLOW #1 

Segmentation on the log-scaled Lysosome signal and aggressively filling holes.



In [5]:

sc_image = intensity_normalization(  img_data[1,:,:,:].copy() ,  scaling_param=[0] )

sc_median = median_filter_slice_by_slice( 
                                                                sc_image,
                                                                size=9 )

log_image, d = log_transform( sc_median ) 
log_image = intensity_normalization(  log_image,  scaling_param=[0] )


edges = filters.scharr(log_image)


gaussian_smoothing_sigma = 4
gaussian_smoothing_truncate_range = 3.0



intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


NameError: name 'log_raw' is not defined

In [16]:

structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   log_image,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )

structure_img = structure_img_smooth + edges

In [29]:
###################
# CORE_PROCESSING
###################

##########################################################################
# DEFAULT PARAMETERS:
#   note that these parameters are supposed to be fixed for the structure
#   and work well accross different datasets

#intensity_norm_param = [0.5, 15]
intensity_norm_param = [0]
gaussian_smoothing_sigma = 5
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100
#structure_img_smooth = raw_gaussian_filter2D
# this is closer to the original 
bw, _bw_low_level = MO(structure_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
                                                


In [30]:

###################
# POST_PROCESSING (gaussian)
###################
width = 80  
# discount z direction
#segmented_soma = remove_small_objects(bw, min_size=width*width*1.5, connectivity=1, in_place=False)
#removed_holes = morphology.remove_small_holes(bw, width ** 3 )
removed_holes = aicssegmentation.core.utils.hole_filling(bw, hole_min =0. , hole_max=width**2, fill_2d = True) 


print(f" remove hole size  ~ : { scale[1]*width} microns, scale:{scale}")

width = 45  
cleaned_img = aicssegmentation.core.utils.size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= width**2, 
                                                         method = "slice_by_slice" ,
                                                         connectivity=1)
print(f" remove small objects  size  ~ : { scale[1]*width} microns, scale:{scale}")

#sobel_image = np.abs(ndi.sobel(struct_img))
# watershed on the sobel limited by the mask
# labels_out = watershed(
#         image=np.abs(ndi.sobel(structure_img_composite)),
#         markers=NU_labels,
#         connectivity=np.ones((3, 3, 3), bool),
#         mask=cleaned_img,
#     )

labels_out = watershed(
        image=edges,
        markers=NU_labels,
        connectivity=np.ones((3, 3, 3), bool),
        mask=cleaned_img,
    )

contour_img = aicssegmentation.core.output_utils.generate_segmentation_contour(cleaned_img)
# def generate_segmentation_contour(im):
#     """generate the contour of the segmentation"""

#     bd = np.logical_xor(erosion(im > 0, selem=ball(1)), im > 0)

#     bd = bd.astype(np.uint8)
#     bd[bd > 0] = 255


 remove hole size  ~ : 6.3897321478698546 microns, scale:(0.5804527163320905, 0.07987165184837318, 0.07987165184837318)
 remove small objects  size  ~ : 3.594224333176793 microns, scale:(0.5804527163320905, 0.07987165184837318, 0.07987165184837318)


In [31]:
labels_simple = label(cleaned_img)
viewer.add_labels(
    labels_simple,
    scale=scale 
)


<Labels layer 'labels_simple [1]' at 0x1706305e0>

Find the label with the brightest florescence..

Keep the Nuclei with that label.  
Mask all the other nuclei.

re-label the soma with watershed using the single.



find the center of mass of each nuclei.  Use those as seeds for the soma...

In [ ]:
# keep the "SOMA" which contains the highest total signal
all_labels = np.unique(labels_out)[1:]

total_flourescence = [ struct_img[labels_out == label].sum() for label in all_labels]
# combine NU and "labels" to make a SOMA
keep_label = all_labels[np.argmax(total_flourescence)]
keep_label

In [22]:

viewer = napari.view_image(
    edges,
    scale=scale,
    blending='additive',
    colormap='magenta',
)

viewer.add_image(
    log_image,
    scale=scale 
)

viewer.add_image(
    sc_image,
    scale=scale 
)

viewer.add_image(
    cleaned_img,
    scale=scale
)
viewer.scale_bar.visible = True

viewer.add_labels(
    labels_out,
    scale=scale 
)


# viewer.add_labels(
#     contour_img,
#     scale=scale 
# )


<Labels layer 'labels_out' at 0x168e07ee0>

In [26]:
viewer.add_labels(
    labels_out2,
    scale=scale 
)


<Labels layer 'labels_out2 [1]' at 0x168bafeb0>

# WORKFLOW #2

In [34]:

###################
# INPUT
###################

composite_channels = [1,4,5,7]
#composite_channels = [1,2,3,4] # andy's best guess but maybe need to scale channel 1 (lysosomes) mega
#composite_channels = range(img_data.shape[0]-1)
composite_channels



gaussian_smoothing_sigma = 8
gaussian_smoothing_truncate_range = 3.0

gaussian_smoothing_sigma_3D = [gaussian_smoothing_sigma*scale[2]/x  for x in scale]



print(f"gaussian filtering width (2D) is ~ : { scale[1]*gaussian_smoothing_sigma} microns, scale:{scale}")
print(f"gaussian filtering width (3d) is ~ : {gaussian_smoothing_sigma_3D} pixels; :{[x*y for x,y in zip(scale,gaussian_smoothing_sigma_3D)]} microns")

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(img_data[1,:,:,:]) #  [0.0, 8]

###################
# PRE_PROCESSING
###################

#truncate_intensity = raw_soma.mean()+raw_soma.std()*3
raw_soma_linear = intensity_normalization(  img_data[composite_channels,:,:,:].copy(), scaling_param=[0] ).sum(axis=0)
raw_soma_linear = intensity_normalization(  raw_soma_linear, scaling_param=[0] )





# non-linear scaling for the aggregate tested below
# # scale_parameters = [[0.0, 24.5],
# #                                     [0.0, 35.0],
# #                                     [0.5, 15.0],
# #                                     [1.5, 10.0]]
# # hold_it = []

struct_img = raw_soma_linear.copy()

# smoothing with gaussian filter
# 3D alternative: image_smoothing_gaussian_3d(...)
gaussian_smoothing_sigma = 3
gaussian_smoothing_truncate_range = 3.0

med_filter_size = 9
#structure_img_median = ndi.median_filter(struct_img,    size=med_filter_size  )

structure_img_median = median_filter_slice_by_slice( 
                                                                struct_img,
                                                                size=9 )

# log_image, d = log_transform( sc_median ) 
# log_image = intensity_normalization(  log_image,  scaling_param=[0] )


structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   structure_img_median,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )

edges = filters.scharr(struct_img)



gaussian filtering width (2D) is ~ : 0.6389732147869854 microns, scale:(0.5804527163320905, 0.07987165184837318, 0.07987165184837318)
gaussian filtering width (3d) is ~ : [1.1008187175429014, 8.0, 8.0] pixels; :[0.6389732147869854, 0.6389732147869854, 0.6389732147869854] microns
mean intensity of the stack: 641.5999045901829
the standard deviation of intensity of the stack: 2144.942904845627
0.9999 percentile of the stack intensity is: 49648.38039997965
minimum intensity of the stack: 0
maximum intensity of the stack: 65535
suggested upper range is 23.0, which is 49975.2867160396
suggested lower range is 0.0, which is 641.5999045901829
So, suggested parameter for normalization is [0.0, 23.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value
intensity normalization: min-max n

NameError: name 'struct_img_scaled' is not defined

In [36]:

# ##########################################################################
# PARAMETERS:

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img) #  [0.5, 12.5]
aicssegmentation.core.pre_processing_utils.suggest_normalization_param(structure_img_smooth) # [0.5, 9.0]
aicssegmentation.core.pre_processing_utils.suggest_normalization_param(structure_img_median) # [0.5, 9.0]
# [0.0, 23.5]

#intensity_norm_param = [0.5, 15]
intensity_norm_param = [0]
gaussian_smoothing_sigma = 5
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100
#structure_img_smooth = raw_gaussian_filter2D
# this is closer to the original 
bw, _bw_low_level = MO(structure_img_smooth, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.25, 
                                                return_object=True,
                                                dilate=True)
                                                




###################
# POST_PROCESSING
###################
width = 5  
# discount z direction
#segmented_soma = remove_small_objects(bw, min_size=width*width*1.5, connectivity=1, in_place=False)
removed_holes = morphology.remove_small_holes(bw, width ** 3 )

width = 6  
cleaned_img = aicssegmentation.core.utils.size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= width**3, 
                                                         method = "3D", #"slice_by_slice" 
                                                         connectivity=1)

#sobel_image = np.abs(ndi.sobel(struct_img))
# watershed on the sobel limited by the mask


labels_out = watershed(
        image=edges, #np.abs(ndi.sobel(structure_img_smooth)),
        markers=NU_labels,
        connectivity=np.ones((3, 3, 3), bool),
        mask=cleaned_img,
    )



mean intensity of the stack: 0.022594163479725156
the standard deviation of intensity of the stack: 0.035176025031434015
0.9999 percentile of the stack intensity is: 0.562098829807297
minimum intensity of the stack: 6.00946325238002e-09
maximum intensity of the stack: 1.0
suggested upper range is 15.5, which is 0.5678225514669525
suggested lower range is 0.5, which is 0.0050061509640081485
So, suggested parameter for normalization is [0.5, 15.5]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value
mean intensity of the stack: 0.02121846708738522
the standard deviation of intensity of the stack: 0.03109671695313
0.9999 percentile of the stack intensity is: 0.49197553144795597
minimum intensity of the stack: 0.005571961371672658
maximum intensity of the stack: 0.7053786193844717


In [37]:

viewer = napari.view_image(
    edges,
    scale=scale,
    blending='additive',
    colormap='magenta',
)

viewer.add_image(
    structure_img_smooth,
    scale=scale 
)

viewer.add_image(
    cleaned_img,
    scale=scale
)
viewer.scale_bar.visible = True

viewer.add_labels(
    labels_out,
    scale=scale 
)



<Labels layer 'labels_out' at 0x171982eb0>

# workflow 3

In [ ]:


# # workflow 1  in cellProfiler "cross-entropy" is Li's method
# li_thresholded = structure_img_smooth > filters.threshold_li(structure_img_smooth)

#sobel1=np.abs(ndi.sobel(structure_img_med_smooth1))
sobel_e=np.abs(ndi.sobel(log_raw))

